In [1]:
!nvidia-smi

Tue May 17 07:41:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers==4.18.0
!pip install datasets

     |████████████████████████████████| 4.0 MB 14.1 MB/s 
     |████████████████████████████████| 596 kB 83.0 MB/s 
     |████████████████████████████████| 84 kB 4.4 MB/s 
     |████████████████████████████████| 880 kB 74.2 MB/s 
     |████████████████████████████████| 6.6 MB 77.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=20579ff957a0e7b274b5e3112879c23068db4d416637cce868624d4ea78e9d39
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 13.6 MB/s 
     |████████████████████████████████| 212 kB 92.1 MB/s 
     |████████████████████████████████| 136 kB 92.4 MB/s 
     |████████████████████████████████| 1.1 MB 71.2 MB/s 
     |████████████████████████████

In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Knowledge Distillation for Question Answering

__Teacher__: `XLM-Roberta-Large`

__Student__: `Multilingual-MiniLM-L12-H384`

In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## Models

In [7]:
from transformers import (AutoModelForQuestionAnswering, AutoTokenizer,
                          Trainer, TrainingArguments)

import collections

In [8]:
batch_size = 32
pretraining_language = 'es'

In [9]:
teacher = AutoModelForQuestionAnswering.from_pretrained("bhavikardeshna/xlm-roberta-base-spanish")
teacher_tokenizer = AutoTokenizer.from_pretrained("bhavikardeshna/xlm-roberta-base-spanish")


student_ckpt = f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
student_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Downloading:   0%|          | 0.00/756 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [10]:
 # Tokenize text and infer from context using QA model
question = "How much music can this hold?"
context = """An MP3 is about 1 MB/minute, so about 6000 hours depending on \
file size."""
inputs = teacher_tokenizer(question, context, return_tensors="pt")
print(teacher_tokenizer.decode(inputs["input_ids"][0]))

<s> How much music can this hold?</s></s> An MP3 is about 1 MB/minute, so about 6000 hours depending on file size.</s>


In [11]:
with torch.no_grad():
    outputs = teacher(**inputs)
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-1.4679, -5.4038, -5.7050, -5.2507, -7.2301, -6.2693, -5.4455, -7.7604,
         -7.2924, -7.3895,  1.3901,  1.4298, -3.1647, -2.0180,  2.6183,  4.7309,
          0.5323, -2.5087, -2.0795, -3.8465, -3.4630, -2.1346,  2.8881,  2.9890,
         -1.8180, -2.8429, -4.9965, -6.0976, -0.7904, -4.2124, -3.6150, -6.9073]]), end_logits=tensor([[-1.7879, -6.8309, -3.5489, -3.5909, -7.0336, -5.6769, -4.9797, -6.3943,
         -6.5106, -6.6107, -2.5300, -2.3652,  0.7986, -3.0972, -1.9265, -0.0312,
          2.5588, -1.5474, -1.6376,  3.5594,  3.6976, -0.8527, -2.3161,  0.4903,
          3.7349, -3.0399, -0.7345, -4.5266, -2.6684, -0.5674,  3.2188, -4.7326]]), hidden_states=None, attentions=None)


## Data

In [12]:
import numpy as np
import pandas as pd
import json

def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [13]:
# training data
language = pretraining_language
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


In [14]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5728d4d3ff5b5019007da7ba,¿Dónde se encuentra el tribunal testamentario del condado de Wayne?,"Los tribunales de Detroit son administrados por el estado y las elecciones no son partidistas. El tribunal testamentario del condado de Wayne está ubicado en el Coleman A. Young Municipal Center en el centro de Detroit. El tribunal de circuito se encuentra al otro lado de la avenida Gratiot. en el Frank Murphy Hall of Justice, en el centro de Detroit. La ciudad alberga el Trigésimo Sexto Tribunal de Distrito, así como el Primer Distrito del Tribunal de Apelaciones de Michigan y el Tribunal de Distrito de los Estados Unidos para el Distrito Este de Michigan. La ciudad proporciona la aplicación de la ley a través del Departamento de Policía de Detroit y servicios de emergencia a través del Departamento de Bomberos de Detroit.",161,Coleman A. Young Municipal Center,0,"{'answer_start': [161], 'text': ['Coleman A. Young Municipal Center']}"


In [15]:
# Validation data
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


,index,question,context,answer_start,text,c_id,answers
0,57373f80c3c5551400e51e91,¿Qué contenían los objetos en movimiento forzado según la idea medieval tardía que influyen en Aristóteles?,"Las deficiencias de la física aristotélica no se corregirían por completo hasta el trabajo del siglo XVII de Galileo Galilei, quien fue influenciado por la idea medieval tardía de que los objetos en movimiento forzado llevaban una fuerza innata de ímpetu. Galileo construyó un experimento en el que las piedras y las balas de cañón fueron rodadas por una pendiente para refutar la teoría aristotélica del movimiento a principios del siglo XVII. Mostró que los cuerpos eran acelerados por la gravedad hasta un punto que era independiente de su masa y argumentó que los objetos retienen su velocidad a menos que actúen por una fuerza, por ejemplo, la fricción.",227,una fuerza innata de ímpetu,0,"{'answer_start': [227], 'text': ['una fuerza innata de ímpetu']}"


### Create Datasets

In [16]:
from datasets import Dataset,DatasetDict


tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [17]:
squad

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 87488
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 34697
    })
})

In [18]:
pad_on_right = student_tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = student_tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(student_tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(
    examples, features, raw_predictions, n_best_size=20, max_answer_length=30
):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
        # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [19]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

## Training

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs,return_outputs=False):
        outputs_stu = model(**inputs)
        # Extract cross-entropy loss and logits from student
        loss_ce = outputs_stu.loss
        # Extract logits from teacher
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            start_logits_tea = outputs_tea.start_logits
            end_logits_tea = outputs_tea.end_logits
        # Soften probabilities and compute distillation loss
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        
        loss_start = loss_fct(
                    nn.functional.log_softmax(outputs_stu.start_logits / self.args.temperature, dim=-1),
                    nn.functional.softmax(start_logits_tea / self.args.temperature, dim=-1),
                ) * (self.args.temperature**2)
        loss_end = loss_fct(
            nn.functional.log_softmax(outputs_stu.end_logits / self.args.temperature, dim=-1),
            nn.functional.softmax(end_logits_tea / self.args.temperature, dim=-1),
        ) * (self.args.temperature**2)
        loss_kd = (loss_start + loss_end) / 2.0

        loss = self.args.alpha * loss_ce + (1 - self.args.alpha) * loss_kd

        return loss

In [21]:
from transformers import default_data_collator


from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    
def student_init():
    return AutoModelForQuestionAnswering.from_pretrained(
        f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
    ).to(device)


student_training_args = DistillationTrainingArguments(
    output_dir = f"./{pretraining_language}-kd-XLM-minilmv2-{batch_size}",
    # evaluation_strategy="epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    temperature = 2.0,
    alpha=0.5,
    warmup_ratio = 0.1,
    fp16=True,
    gradient_accumulation_steps = 8,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit=1,
    # load_best_model_at_end=True
)

teacher.to(device)


data_collator = default_data_collator

distil_trainer = DistillationTrainer(
    model_init=student_init,
    teacher_model=teacher,
    args=student_training_args,
    train_dataset=squad["train"],
    # eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=student_tokenizer,
    # compute_metrics=compute_metrics,
)


https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjetuhwsw


Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

storing https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/54765531012f342002b6e58109adb9698d419d2cccace50841815ef3aecf3c79.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
creating metadata file for /root/.cache/huggingface/transformers/54765531012f342002b6e58109adb9698d419d2cccace50841815ef3aecf3c79.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
loading configuration file https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/54765531012f342002b6e58109adb9698d419d2cccace50841815ef3aecf3c79.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/es-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hid

Downloading:   0%|          | 0.00/450M [00:00<?, ?B/s]

storing https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/bd8ae21ad3a5adaf93071457acfcbc20a98e4033e84c73e28cd487db20c82482.69f0039359da0325e9c4a398d1f12adbb25b22ef0be72b3a7fea7f29581e9da3
creating metadata file for /root/.cache/huggingface/transformers/bd8ae21ad3a5adaf93071457acfcbc20a98e4033e84c73e28cd487db20c82482.69f0039359da0325e9c4a398d1f12adbb25b22ef0be72b3a7fea7f29581e9da3
loading weights file https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/bd8ae21ad3a5adaf93071457acfcbc20a98e4033e84c73e28cd487db20c82482.69f0039359da0325e9c4a398d1f12adbb25b22ef0be72b3a7fea7f29581e9da3
Some weights of the model checkpoint at subhasisj/es-TAPT-MLM-MiniLM were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictio

In [22]:
distil_trainer.train()

loading configuration file https://huggingface.co/subhasisj/es-TAPT-MLM-MiniLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/54765531012f342002b6e58109adb9698d419d2cccace50841815ef3aecf3c79.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/es-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

load

Step,Training Loss
500,4.287500
1000,2.246300
1500,1.541200


Saving model checkpoint to ./es-kd-XLM-minilmv2-32/checkpoint-356
Configuration saved in ./es-kd-XLM-minilmv2-32/checkpoint-356/config.json
Model weights saved in ./es-kd-XLM-minilmv2-32/checkpoint-356/pytorch_model.bin
tokenizer config file saved in ./es-kd-XLM-minilmv2-32/checkpoint-356/tokenizer_config.json
Special tokens file saved in ./es-kd-XLM-minilmv2-32/checkpoint-356/special_tokens_map.json
tokenizer config file saved in ./es-kd-XLM-minilmv2-32/tokenizer_config.json
Special tokens file saved in ./es-kd-XLM-minilmv2-32/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
Saving model checkpoint to ./es-kd-XLM-minilmv2-32/checkpoint-712
Configuration saved in ./es-kd-XLM-minilmv2-32/checkpoint-712/config.json
Model weights saved in ./es-kd-XLM-minilmv2-32/checkpoint-712/pytorch_model.bin
tokenizer config file saved in ./es-kd-XLM-minilmv2-32/checkpoint-712/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=1780, training_loss=2.4789748288272473, metrics={'train_runtime': 10516.7735, 'train_samples_per_second': 43.318, 'train_steps_per_second': 0.169, 'total_flos': 2.235180663910656e+16, 'train_loss': 2.4789748288272473, 'epoch': 5.0})

In [23]:
distil_trainer.push_to_hub("Training Completed")

Saving model checkpoint to ./es-kd-XLM-minilmv2-32
Configuration saved in ./es-kd-XLM-minilmv2-32/config.json
Model weights saved in ./es-kd-XLM-minilmv2-32/pytorch_model.bin
tokenizer config file saved in ./es-kd-XLM-minilmv2-32/tokenizer_config.json
Special tokens file saved in ./es-kd-XLM-minilmv2-32/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/448M [00:00<?, ?B/s]

Upload file runs/May17_07-45-54_7af744249a5b/events.out.tfevents.1652773620.7af744249a5b.81.0:  81%|########1 …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/es-kd-XLM-minilmv2-32
   3a80521..b750e33  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/es-kd-XLM-minilmv2-32
   b750e33..dea4381  main -> main



'https://huggingface.co/subhasisj/es-kd-XLM-minilmv2-32/commit/b750e33f636dd3d127c7e5b72bbb818cd17b1fde'